### CMIP6 Data Issues 
1. The ES-DOC errata pages are used for modelling centers to report issues with their published data here:
      [ES-DOC ERRATA]( https://errata.es-doc.org )
2. A separate list of exceptions is kept as we process the data (concatenating netcdf and saving as zarr)
      [ESGF to GCS Issues]( https://docs.google.com/spreadsheets/d/e/2PACX-1vRxKgz1xCH7zhUoDnl_llgEvbj2ssxoJiTUdbkHkkfWiCKU8EfZtPerar3ELjoIzAda5giR06QvbWGE/pubhtml?gid=128595157&single=true )
3. Issues with the existing Google Cloud collection are crowd sourced here:
      [GCS Issues]( https://tinyurl.com/y5cw76at )

### This notebook updates the list of processing exceptions in Issue 2.

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime
from ast import literal_eval
import os
import numpy as np
import os
gspread.__version__

'3.1.0'

In [2]:
json_keyfile = '/home/naomi/cmip6-zarr/json/Pangeo Hackathon-e48a41b13c91.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)

In [3]:
sheet_name = "CMIP6_DataExceptions (Responses)"
sh = gc.open(sheet_name)
print(sh.worksheets())

[<Worksheet 'Form Responses 1' id:252604281>, <Worksheet 'NH_additions' id:128595157>]


In [4]:
wks = sh.worksheet("NH_additions")

data = wks.get_all_values()
headers = data.pop(0)

df_cloud = pd.DataFrame(data, columns=headers)
#df_cloud['name'] = ['/'.join(s[:-3]) for s in df_cloud.values]
df_local = pd.read_csv('csv/exceptions.csv', na_filter= False)
#df_local['name'] = ['/'.join(s[:-3]) for s in df_local.values]

# add new from df_local
for item,row in enumerate(df_local.values):
    name = row[-1]
    #print(item,name)
    df_match = df_cloud[df_cloud.name==name]
    if len(df_match)==0:
        print(item,name,' is not in df_cloud')
        wks.append_row(list(row))

64 CNRM-CM6-1/historical/r21i1p1f2/Omon/obvfsq/gn/noUse  is not in df_cloud
218 GISS-E2-1-G/historical/r7i1p3f1/Lmon/mrro/gn/noUse  is not in df_cloud
219 GISS-E2-1-G/historical/r7i1p3f1/Lmon/mrso/gn/noUse  is not in df_cloud
220 GISS-E2-1-G/historical/r7i1p3f1/Lmon/mrsos/gn/noUse  is not in df_cloud
221 GISS-E2-1-G/historical/r7i1p3f1/LImon/all/gn/noUse  is not in df_cloud
294 ACCESS-ESM1-5/piControl/r1i1p1f1/Omon/umo/gn/noUse  is not in df_cloud


In [5]:
wks = sh.worksheet("NH_additions")

data = wks.get_all_values()
headers = data.pop(0)

df_cloud = pd.DataFrame(data, columns=headers)
#df_cloud['name'] = ['/'.join(s[:-3]) for s in df_cloud.values]
df_local = pd.read_csv('csv/exceptions.csv', na_filter= False)
#df_local['name'] = ['/'.join(s[:-3]) for s in df_local.values]

# delete old from df
for item,row in enumerate(df_cloud.values):
    name = row[-1]
    #print(item,name)
    df_match = df_local[df_local.name==name]
    if len(df_match)==0:
        print(item,name,' is not in df_local')
        wks.delete_row(item+2)

In [6]:
assert False 

AssertionError: 

In [20]:
# add a new local entry
import datetime
date = str(datetime.datetime.now().strftime("%Y-%m-%d"))

store = 'CESM2-WACCM/piControl/r1i1p1f1/Oyr/bddtalk/gn/'
code = 'noUse'
reason = 'trouble with grid - missing_value'

row = store.split('/')[:-1] + [code,reason,date,store+code]
print(row)

['CESM2-WACCM', 'piControl', 'r1i1p1f1', 'Oyr', 'bddtalk', 'gn', 'noUse', 'trouble with grid - missing_value', '2020-03-01', 'CESM2-WACCM/piControl/r1i1p1f1/Oyr/bddtalk/gn/noUse']


In [21]:
# if it looks good, now check
dfl = df_local.append(pd.Series(row, index=df_local.columns ),ignore_index=True)
dfl.tail()

,source_id,experiment_id,member_id,table_id,variable_id,grid_label,reason_code,reason_txt,status,name
291,CESM2,ssp126,r2i1p1f1,day,tasmin,gn,noUse,some data missing,2020-02-10,CESM2/ssp126/r2i1p1f1/day/tasmin/gn/noUse
292,CESM2,ssp370,r2i1p1f1,day,tasmax,gn,noUse,multiple time gaps,2020-02-10,CESM2/ssp370/r2i1p1f1/day/tasmax/gn/noUse
293,MRI-ESM2-0,ssp585,r1i1p1f1,CFmon,albisccp,gn,noUse,multiple time gaps,2020-02-10,MRI-ESM2-0/ssp585/r1i1p1f1/CFmon/albisccp/gn/n...
294,ACCESS-ESM1-5,piControl,r1i1p1f1,Omon,umo,gn,noUse,HDF error,2020-02-16,ACCESS-ESM1-5/piControl/r1i1p1f1/Omon/umo/gn/n...
295,CESM2-WACCM,piControl,r1i1p1f1,Oyr,bddtalk,gn,noUse,trouble with grid - missing_value,2020-03-01,CESM2-WACCM/piControl/r1i1p1f1/Oyr/bddtalk/gn/...


In [22]:
dfl.to_csv('csv/exceptions.csv', index=False)

In [23]:
assert False

AssertionError: 

In [ ]:
#wks.update_acell('B1', 'Bingo!')
#wks.append_row(['junk','more_junk'])
#wks.add_rows(2)
#for row in df_local.values[:100]:
#    wks.append_row(list(row))
#wks.delete_row(7)
#wks.row_values(8)

In [18]:
common_cols = df_cloud.columns.tolist()                              #generate list of column names
df12 = pd.merge(df_cloud, df_local, on=common_cols, how='inner')     #extract common rows with merge
df2 = df_local[~df_local['name'].isin(df12['name'])]
df1 = df_cloud[~df_cloud['name'].isin(df12['name'])]
len(df_cloud),len(df_local)

(282, 282)